In [ ]:
"""
import packages
"""
import os, glob
import pandas as pd
import numpy as np
import math, random
from datetime import datetime
import pickle
from matplotlib import pyplot as plt
import argparse
import path, sys, re, time
from collections import Counter
from scipy.signal import find_peaks

In [ ]:
"""
import custom modules
"""
from module_.readText import read_hh
from module_.featureExtraction import feature_extraction
from module_.changePointDetection import change_point_detection
from module_.info.hh101_info import baseline_activities

# hh101 (CASAS, 1-resident, serialized)

In [ ]:
"""
0. load casas dataset: hh101
"""
with open('dataset/hh/hh101/ann.txt','rb') as f: 
    rawdata=f.readlines()
events=read_hh(rawdata)
events=np.array(events)
print(events.shape)

In [ ]:
"""
1. remove all except M and D
"""
events_md=np.array([event for event in events if event[0][0] in ['M', 'D']])
print(events_md.shape)

trs_md=[i for i in range(len(events_md)) if events_md[i][3]!=events_md[max(i-1,0)][3]]
print(len(trs_md))

for i in range(events_md.shape[0]):
    events_md[i,3]='Idle' if events_md[i,3]=="" else events_md[i,3]

episodes, trs, tags = [], [], []
previdx=0
for i in range(len(trs_md)-1):
    # left activity
    # right activity
    left=np.array(events_md[previdx:trs_md[i]])
    right=np.array(events_md[trs_md[i]:trs_md[i+1]])
    episode=np.concatenate((left, right))
    # print(episode.shape)
    episodes.append(episode)
    trs.append(left.shape[0])
    tags.append("{}-{}".format(left[0][3], right[0][3]))

    previdx=trs_md[i]

In [ ]:
"""make combinations
    1. pick one group type
    2. pick an activity stream from the group
    3. pick another group type w/o type 1.
    4. pick an activity stream from the group
"""

data_name='hh101'
metric='SEP'

for i, eps in enumerate(episodes):
    print(i, tags[i])
    sensor_list=sorted(set(eps[:,0]))
    features=feature_extraction(eps, data_name, sensor_list)
    scores=change_point_detection(features, i, tags[i], data_name=data_name, metric=metric, save=True)
    scores=np.array(scores)
    scores[scores<0]=0

    peak, _ =find_peaks(scores)
    positive=[i for i in peak if scores[i]>0.45]
    
###
    plt.title("{}-{}".format(tags[i], i))
    plt.ylabel('score')
    plt.xlabel('event')
    plt.ylim(0,0.7)
    plt.bar(range(len(eps)), scores)
    plt.plot(positive, np.array(scores)[positive], 'bx', label='peak')
    plt.axhline(y=0.3, linestyle=':', color='r', label='threshold')
    plt.axvline(x=trs[i], linestyle=':', color='g', label='transition')
    plt.legend()
    plt.savefig("./outputs/{}/{}/{}/{}/graph.png".format(data_name, metric, tags[i], i))
    plt.clf()
###

In [11]:
"""
    hh101 Evaluation
    - load scores
"""
total_counts=np.zeros(4)
denom = numer = 0
for activity_folder in glob.glob("./outputs/{}/{}/*".format(data_name, metric)):
    # one type of pairs
    activity_pair=activity_folder.split("/")[-1]
    print(activity_pair)
    pair_counts=np.zeros(4) # TP, FP, TN, FN

    for episode_folder in glob.glob("{}/*".format(activity_folder)):
        denom+=1
        eps_order=int(episode_folder.split("/")[-1])
        eps, point=episodes[eps_order], trs[eps_order]
        scores=np.load("{}/scores.npy".format(episode_folder))

        peaks, _ = find_peaks(scores)
        positives=[i for i in peaks if scores[i]>0.1]
        numer+=len(positives)
        # positives=[i for i in range(len(scores)) if scores[i]>0.3]
        ttimestamp=float(eps[point][2])

        for i in range(len(scores)):
            if i in positives:
                if i==point:
                    pair_counts[0]+=1
                else:
                    timestamp_b=float(eps[i-1][2])
                    timestamp_a=float(eps[i][2])
                    if abs(ttimestamp-timestamp_b)<10 or abs(ttimestamp-timestamp_a)<10:
                        pair_counts[0]+=1
                    else:
                        pair_counts[1]+=1
            else:
                if i==point:
                    pair_counts[3]+=1
                else:
                    pair_counts[2]+=1
    TPR_=pair_counts[0]/(pair_counts[0]+pair_counts[3])
    FPR_=pair_counts[1]/(pair_counts[1]+pair_counts[2])
    print("Avg. TPR and FPR: ({}, {})".format(TPR_, FPR_))

    total_counts+=pair_counts

TPR=total_counts[0]/(total_counts[0]+total_counts[3])
FPR=total_counts[1]/(total_counts[1]+total_counts[2])
print("Total Avg. TPR and FPR: ({}, {})".format(TPR, FPR))

print(numer/denom)




Wash_Dishes-Evening_Meds
Avg. TPR and FPR: (0.6666666666666666, 0.0784313725490196)
Watch_TV-Wash_Dinner_Dishes
Avg. TPR and FPR: (0.75, 0.13353115727002968)
Idle-Phone
Avg. TPR and FPR: (0.6236559139784946, 0.12389380530973451)
Cook_Dinner-Idle
Avg. TPR and FPR: (0.691358024691358, 0.11080958842152872)
Enter_Home-Personal_Hygiene
Avg. TPR and FPR: (0.6666666666666666, 0.1590909090909091)
Sleep_Out_Of_Bed-Relax
Avg. TPR and FPR: (0.3333333333333333, 0.08264462809917356)
Watch_TV-Wash_Breakfast_Dishes
Avg. TPR and FPR: (1.0, 0.12396694214876033)
Wash_Dishes-Watch_TV
Avg. TPR and FPR: (0.0, 0.11313868613138686)
Evening_Meds-Wash_Dinner_Dishes
Avg. TPR and FPR: (1.0, 0.12318840579710146)
Idle-Dress
Avg. TPR and FPR: (0.632996632996633, 0.11318773624527509)
Eat_Lunch-Idle
Avg. TPR and FPR: (0.5, 0.1111111111111111)
Dress-Idle
Avg. TPR and FPR: (0.5365853658536586, 0.11263064265065599)
Read-Idle
Avg. TPR and FPR: (0.4444444444444444, 0.0784313725490196)
Phone-Idle
Avg. TPR and FPR: (0.68055